Дана база покупок продуктового магазина:

    id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
    Товар – наименование товара
    Количество – число единиц купленного товара

Задача: выяснить, какие пары товаров пользователи чаще всего покупают вместе. По сути, необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/409319/test1_completed.csv')
df.shape

(43514, 3)

In [3]:
# избавимся от пустых значений 

df = df.dropna()
df.shape

# таких правда не оказалось, размер датафрейма не изменился

(43514, 3)

In [4]:
# проверим, есть ли случаи, когда в одной покупке два раза присутствует одна и та же позиция
# сравним количество уникальных позиций в каждой покупке с общим количеством позиций в каждой покупке

poisk_duplikatov = df.groupby('id').agg({'Товар': pd.Series.nunique}) == df.groupby('id').agg({'Товар': 'count'}) 
poisk_duplikatov.loc[poisk_duplikatov['Товар'] == False]

# таких покупок нет

,Товар
id,


In [5]:
# посмотрим, есть ли отрицательные или нулевые покупки по количеству

df.loc[df['Количество'] <= 0]

# таких покупок нет

,id,Товар,Количество


In [6]:
# теперь перейдем к подсчету пар
# отсортируем товары по номерам покупок и товарам в алфавитном порядке

df = df.sort_values(['id', 'Товар'])

# далее сформируем датафрейм "покупка - перечень товаров в ней в алфавитном порядке" 

po_zakazam = df.groupby('id')\
                      .agg({'Товар': lambda x:list(x)})

In [7]:
# пробежимся по каждой покупке циклом и запишем все возможные пары

pairs = []
for i in po_zakazam['Товар']:
        for j in range(len(i)):
            for k in range(j+1,len(i)):
                pairs.append([i[j],i[k]])
                
# сформируем датафрейм из наших пар  и присвоим каждой встречаемость 1
                
pairs_df = pd.DataFrame(pairs)
pairs_df['Встречаемость'] = 1
pairs_df.rename(columns={0:'1_Товар',1:'2_Товар'},inplace=True)

In [8]:
# сделаем агрегацию по парам и отсортируем

occurrence_pairs = pairs_df.groupby(['1_Товар','2_Товар']).agg({'Встречаемость':'sum'})

occurrence_pairs = occurrence_pairs.sort_values('Встречаемость',ascending=False)

In [9]:
#  смотрим 10 наиболее распространенных паттернов

occurrence_pairs.head(10)

,,Встречаемость
1_Товар,2_Товар,
Огурцы Луховицкие,Укроп,431
Петрушка,Укроп,408
Арбуз,Огурцы Луховицкие,345
Кабачки,Огурцы Луховицкие,326
Кинза,Укроп,303
Лук зеленый,Укроп,300
Огурцы Луховицкие,Петрушка,286
Лук репчатый,Огурцы Луховицкие,285
Баклажаны грунтовые,Кабачки,284


---
Будет интересно еще посмотреть, какая доля покупок определенного товара попадает на конкретную пару.
Вычислим товары, для которых такие пары наиболее встречаются

In [10]:
# для дальнейших расчетов посчитаем количество покупок каждого товара 
pokupki_po_tovaram = df['Товар'].value_counts()

In [11]:
# теперь посчитаем процент каждого товара, приходящийся на наличие конкретной пары, 
# от общего числа покупок данного товара

occurrence_pairs = occurrence_pairs.reset_index()

occurrence_pairs['Процент_1_товар'] = (
                                      (occurrence_pairs['Встречаемость'] /
                                      occurrence_pairs['1_Товар'].apply(lambda x: pokupki_po_tovaram[x]))
                                      .apply(lambda x: int(x * 100))
                                      )

occurrence_pairs['Процент_2_товар'] = (
                                      (occurrence_pairs['Встречаемость'] /
                                      occurrence_pairs['2_Товар'].apply(lambda x: pokupki_po_tovaram[x]))
                                      .apply(lambda x: int(x * 100))
                                      )

In [12]:
# для большей читабельности  преобразуем наш массив

# сделаем 2 массива - каждый только с одним главным товаром(крупными буквами), указанием пары(мелкими буквами) 
# и долей для этой пары от общего числа покупок главного товара

occurrence_pairs_1 = occurrence_pairs
occurrence_pairs_2 = occurrence_pairs

occurrence_pairs_1['Пара'] = (
                              occurrence_pairs_1['1_Товар'].str.upper() + '>>>' +  
                               occurrence_pairs_1['2_Товар'].str.lower()
                              )
occurrence_pairs_1 = occurrence_pairs_1.rename(columns={'Процент_1_товар':'Процент товара в паре от общего количества'})
occurrence_pairs_1 = occurrence_pairs_1.drop(columns=['1_Товар','2_Товар','Встречаемость','Процент_2_товар'])
occurrence_pairs_1 = occurrence_pairs_1[['Пара','Процент товара в паре от общего количества']]

occurrence_pairs_2['Пара'] = (
                              occurrence_pairs_2['2_Товар'].str.upper() + '>>>' +  
                               occurrence_pairs_2['1_Товар'].str.lower()
                              )
occurrence_pairs_2 = occurrence_pairs_2.rename(columns={'Процент_2_товар':'Процент товара в паре от общего количества'})
occurrence_pairs_2 = occurrence_pairs_2.drop(columns=['1_Товар','2_Товар','Встречаемость','Процент_1_товар'])
occurrence_pairs_2 = occurrence_pairs_2[['Пара','Процент товара в паре от общего количества']]

In [13]:
# получилось 2 массива со всеми возможными парами
occurrence_pairs_1.head()

,Пара,Процент товара в паре от общего количества
0,ОГУРЦЫ ЛУХОВИЦКИЕ>>>укроп,42
1,ПЕТРУШКА>>>укроп,73
2,АРБУЗ>>>огурцы луховицкие,35
3,КАБАЧКИ>>>огурцы луховицкие,49
4,КИНЗА>>>укроп,62


In [14]:
occurrence_pairs_2.head()

,Пара,Процент товара в паре от общего количества
0,УКРОП>>>огурцы луховицкие,52
1,УКРОП>>>петрушка,49
2,ОГУРЦЫ ЛУХОВИЦКИЕ>>>арбуз,33
3,ОГУРЦЫ ЛУХОВИЦКИЕ>>>кабачки,31
4,УКРОП>>>кинза,36


In [15]:
# объединим 2 этих массива и отсортируем

dolya_pary = (
                pd.concat([occurrence_pairs_1,occurrence_pairs_2])
                .sort_values('Процент товара в паре от общего количества',ascending=False)
             )

In [16]:
# посмотрим на результат
dolya_pary.head(20)

,Пара,Процент товара в паре от общего количества
1,ПЕТРУШКА>>>укроп,73
5,ЛУК ЗЕЛЕНЫЙ>>>укроп,64
4,КИНЗА>>>укроп,62
0,УКРОП>>>огурцы луховицкие,52
80,РЕДИС>>>огурцы луховицкие,52
16,КИНЗА>>>огурцы луховицкие,51
6,ПЕТРУШКА>>>огурцы луховицкие,51
13,ПОМИДОРЫ ПАРАДАЙЗ>>>огурцы луховицкие,50
272,САЛАТ МИКС>>>огурцы луховицкие,50
3,КАБАЧКИ>>>огурцы луховицкие,49


---
Интересно, например, что укроп сопутствует петрушке в покупке почти в 3/4 случаев,
а вот петрушка укропу только в половине.